In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import os
import numpy as np
import pandas as pd
import cv2
import random
import matplotlib.pyplot as plt
%matplotlib inline

## 图片重新命名

In [ ]:
import os

#给所有这个文件夹下的文件自动按数字命名
path = "/Users/XuanQuan/code/ML-Project/dataset/Vtuber_Kizuna-AI"

num = 1
for file in os.listdir(path):
    #如果是jpg文件
    if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
        #重命名
        os.rename(os.path.join(path, file), os.path.join(path, str(num) + '.jpg'))
        num += 1

## 改变图片大小

In [17]:
# 原始图片文件夹路径
ori_path = '/Users/XuanQuan/code/ML-Project/test-dataset/Tsumugi-Kotobuki/'

# 修改后保存的文件夹路径
save_path = '/Users/XuanQuan/code/ML-Project/test-dataset-resized/43-F-M-Is the Order a Rabbit-Syaro Kirima/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
for pathname, dirnames, filenames in os.walk('./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/'):
    for filename in filenames:
        if filename.endswith('.jpg') or filename.endswith('.png'):
            print(pathname + filename)
            img = cv2.imread(pathname + filename)
            resized = cv2.resize(img, (96, 96), interpolation=cv2.INTER_AREA)
            #write the img to the new folder
            cv2.imwrite(save_path + filename, resized)

./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/88.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/77.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/63.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/62.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/76.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/89.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/60.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/74.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/48.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/49.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/75.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/61.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/59.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/65.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/71.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/70.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/64.png
./raw/43-F-M-Is the Order a Rabbit-Syaro Kirima/58.png
./raw/43-F

## 数据增广Test

In [ ]:
import imgaug as ia

from imgaug import augmenters as iaa

import numpy as np
import re
import cv2
import tensorflow as tf
import os.path
def image_augmentation(file_name):
	base_name = os.path.basename(file_name)
	img= cv2.imread(file_name)
	sometimes = lambda aug: iaa.Sometimes(0.5, aug) #建立lambda表达式，
	seq = iaa.Sequential(
    [
        iaa.Fliplr(0.5), # 对50%的图像进行镜像翻转
        sometimes(iaa.Crop(percent=(0, 0.1))), 
	   #对随机的一部分图像做crop操作
       # crop的幅度为0到10%
        sometimes(iaa.Affine(                          #对一部分图像做仿射变换
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},#图像缩放为80%到120%之间
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, #平移±20%之间
            rotate=(-45, 45),   #旋转±45度之间
            shear=(-16, 16),    #剪切变换±16度，（矩形变平行四边形）
            order=[0, 1],   #使用最邻近差值或者双线性差值
            cval=(0, 255),  #全白全黑填充
            mode=ia.ALL    #定义填充图像外区域的方法
        )),
        # 使用下面的0个到4个之间的方法去增强图像。
        iaa.SomeOf((0, 4),
            [
                #用高斯模糊，均值模糊，中值模糊中的一种增强
                iaa.OneOf([
                  	iaa.GaussianBlur((0, 3.0)),
                    iaa.AverageBlur(k=(2, 7)), # 核大小2~7之间，k=((5, 7), (1, 3))时，核高度5~7，宽度1~3
                    iaa.MedianBlur(k=(3, 11)),
                ]),
                #锐化处理
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
                #浮雕效果
               	iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),
                iaa.AdditiveGaussianNoise(
                    loc=0, scale=(0.0, 0.05*255), per_channel=0.5
                ),
                # 将整个图像的对比度变为原来的一半或者二倍
                iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5),
                #把像素移动到周围的地方。
                sometimes(
                    iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)
                ),
                # 扭曲图像的局部区域
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05)))
            ],
            random_order=True # 随机的顺序把这些操作用在图像上
            )
        ],
         random_order=True # 随机的顺序把这些操作用在图像上
       )
	images_aug = [seq.augment_image(img) for _ in range(64)]#一张图片通过数据增广多出来64张
	for i in range(64):
		dir_file=re.sub(r'\..*$', '', file_name)+'_'+str(i)#提取图片后缀之前的名称，并在原名称后添加i表示是增广的第i张图片
		cv2.imwrite(dir_file+os.path.splitext(file_name)[-1],images_aug[i])
